In [24]:
import pandas as pd

In [41]:
#Creating a merged dataframe for the three models outputs

lrdf = pd.read_csv("Tai_Logistic_Regression/Churn Risk.csv")
rfdf = pd.read_csv("Random Forest/RF_churn_risk.csv")
df = rfdf.merge(lrdf, on="CustomerID")
df.rename(columns={"Churn Probability":"RF Churn Risk", "prob_churn":"Tai LR Churn Risk"}, inplace=True)
sylrdf = pd.read_csv("SumYe_Logistic_Regression/churn_risk.csv")
sylrdf.rename(columns={"Churn Probability":"SumYe LR Churn Risk"}, inplace=True)
df = df.merge(sylrdf, on="CustomerID")
df.head()

,CustomerID,RF Churn Risk,Tai LR Churn Risk,SumYe LR Churn Risk
0,2137-DQMEV,0.091893,0.057343,0.017952
1,4636-TVXVG,0.026269,0.038912,0.009673
2,2480-JZOSN,0.556399,0.396576,0.584238
3,9050-IKDZA,0.815906,0.771874,0.822181
4,9397-TZSHA,0.054608,0.037140,0.009965


In [43]:
#Working out the average distance between the three model's predictions

df["min"] = df.loc[:,"RF Churn Risk":"SumYe LR Churn Risk"].min(axis=1)
df["max"] = df.loc[:,"RF Churn Risk":"SumYe LR Churn Risk"].max(axis=1)
df["gap"] = df["max"] - df["min"]
df.gap.mean()

0.10097740122916136

In [44]:
#Creating an average of the three probability predictions weighted by the recall of each model

df["combined"] = (0.7771 * df["RF Churn Risk"] + 0.8214 * df["SumYe LR Churn Risk"] + 0.8112 * df["Tai LR Churn Risk"]) / 2.4097
df = df.sort_values("combined", ascending=False)


In [45]:
#Creating a new top 500 based on this combined probability prediction
combined_top_500 = list(df.iloc[:500]["CustomerID"])


In [46]:
#Getting the ids listed by Sum-Ye's model's top 500 - the best performing model so far

sumye_top_500 = []
with open("SumYe_Logistic_Regression/most_likely_churners.txt", "r") as file:
    for id in file:
        id = id.strip()
        sumye_top_500.append(id)

In [49]:
#Seeing how many ids would be replaced
new_ids = len([id for id in combined_top_500 if id not in sumye_top_500])
new_ids

60

In [20]:
#Writing the new top 500 to a text file
with open("combined_top_500.txt", "w") as file:
    for id in combined_top_500:
        file.write(f"{id}\n")

In [23]:
#Writing the combined probabilities to a csv
df[["CustomerID",'combined']].to_csv("combined_churn_risk.csv")